In [1]:
import doutils
doutils.nb.megaimport(globals())
%load_ext autoreload
%autoreload 2

In [2]:
patterns = "carto-do.safegraph.humanmobility_patterns_usa_latlon_v3_daily_v3"
geography = "carto-do.safegraph.geography_usa_latlon_v3"

bq_client = get_bq_client()

In [3]:
from shapely.geometry import box, Polygon
import h3
from cartoframes.viz import Map, Layer, size_bins_style, color_bins_style, color_continuous_style, basic_style

from sf_utils import *

In [5]:
bboxes = {
    "chicago" : (-87.9574,41.661,-87.5412,42.04),
    "los_angeles" : (-118.507,33.66,-117.78, 34.148),
    "buffalo" : (-78.985778,42.723637,-78.67013,43.008951),
    "raleigh" : (-78.722225,35.715583,-78.558165,35.849233),
}

In [4]:
for k,v in bboxes.items():
    print(k, box(*v).area)

chicago 0.15773980000000037
los_angeles 0.3547760000000067
buffalo 0.09005879347200096
raleigh 0.02192661899999826


In [ ]:
jobs = {}
for city, bbox in bboxes.items():
    q = f"""
        CREATE OR REPLACE TABLE `cartodb-on-gcp-datascience.juanluis.safegraph_blackfriday_weekly_patterns_{city}` AS 
            WITH categories as (
                SELECT geoid, top_category, sub_category, latitude, longitude
                FROM `carto-do.safegraph.pointsofinterest_coreplaces_usa_latlon_v3_daily_v2` 
            )

            SELECT * except(geoid),
                `bqcarto`.h3.LONGLAT_ASH3(longitude, latitude, 7) as h3_z7,  
                `bqcarto`.h3.LONGLAT_ASH3(longitude, latitude, 8) as h3_z8,
                `bqcarto`.h3.LONGLAT_ASH3(longitude, latitude, 9) as h3_z9,
                `bqcarto`.quadkey.LONGLAT_ASQUADINT(longitude, latitude, 15) as qk_z15,
                `bqcarto`.quadkey.LONGLAT_ASQUADINT(longitude, latitude, 17) as qk_z17,
                `bqcarto`.quadkey.LONGLAT_ASQUADINT(longitude, latitude, 18) as qk_z18,
            FROM `cartodb-on-gcp-datascience.juanluis.safegraph_patterns` d
            JOIN categories
            ON  d.placekey = categories.geoid
            WHERE ST_INTERSECTSBOX(ST_GEOGPOINT(longitude, latitude), {str(bbox)[1: - 1]})
        """
    job = bq_client.query(q)
    jobs[city] = job

for city, job in jobs.items():
    try:
        job.result()
        print(f"Job for {city} succeeded")
    except Exception as e:
        print(f"Job for {city} failed", e)

In [ ]:
jobs = {}
for city, bbox in bboxes.items():
    q = f"""
        CREATE OR REPLACE TABLE `cartodb-on-gcp-datascience.juanluis.safegraph_blackfriday_{city}` AS 
            WITH categories as (
                SELECT geoid, top_category, sub_category
                FROM `carto-do.safegraph.pointsofinterest_coreplaces_usa_latlon_v3_daily_v2` 
            )

            SELECT * except(geoid),
                bqcarto.h3.LONGLAT_ASH3(longitude, latitude, 7) as h3_z7,  
                bqcarto.h3.LONGLAT_ASH3(longitude, latitude, 8) as h3_z8,
                bqcarto.h3.LONGLAT_ASH3(longitude, latitude, 9) as h3_z9,
                bqcarto.quadkey.LONGLAT_ASQUADINT(longitude, latitude, 15) as qk_z15,
                bqcarto.quadkey.LONGLAT_ASQUADINT(longitude, latitude, 17) as qk_z17,
                bqcarto.quadkey.LONGLAT_ASQUADINT(longitude, latitude, 18) as qk_z18,
            FROM `carto-do.safegraph.humanmobility_patterns_usa_latlon_v3_daily_v3` d
            JOIN categories
            ON  d.geoid = categories.geoid
            JOIN `carto-do.safegraph.geography_usa_latlon_v3` g
            ON g.geoid = d.geoid
            WHERE ST_INTERSECTSBOX(ST_GEOGPOINT(longitude, latitude), {str(bbox)[1: - 1]})
        """
    job = bq_client.query(q)
    jobs[city] = job

for city, job in jobs.items():
    try:
        job.result()
        print(f"Job for {city} succeeded")
    except Exception as e:
        print(f"Job for {city} failed", e)

### Upload weekly patterns to BigQuery

In [630]:
blobs = list_blobs("gs://carto-safegraph/carto/weekly_sample/patterns", "uris")

In [636]:
SCHEMA = {'placekey': 'STRING',
          'parent_placekey': 'STRING',
          'location_name': 'STRING',
          'street_address': 'STRING',
          'city': 'STRING',
          'region': 'STRING',
          'postal_code': 'INTEGER',
          'iso_country_code': 'STRING',
          'safegraph_brand_ids': 'STRING',
          'brands': 'STRING',
          'date_range_start': 'TIMESTAMP',
          'date_range_end': 'TIMESTAMP',
          'raw_visit_counts': 'INTEGER',
          'raw_visitor_counts': 'INTEGER',
          'visits_by_day': 'STRING',
          'visits_by_each_hour' : 'STRING',
          'poi_cbg': 'STRING',
          'visitor_home_cbgs': 'STRING',
          'visitor_home_aggregation': 'STRING',
          'visitor_daytime_cbgs': 'STRING',
          'visitor_country_of_origin': 'STRING',
          'distance_from_home': 'INTEGER',
          'median_dwell': 'FLOAT',
          'bucketed_dwell_times': 'STRING',
          'related_same_day_brand': 'STRING',
          'device_type': 'STRING',
          'tcarrier_name' : 'STRING'
            }

In [637]:
schema = [ {"name" : k, "type" : v} for k,v in SCHEMA.items()
    
]

In [638]:
job_config = get_jobconfig_brute(other_params={"schema" : schema})

In [639]:
bq_client = get_bq_client(gcloud_project="cartodb-on-gcp-datascience")

In [ ]:
job = load_blobs_to_bq_job(
    "cartodb-on-gcp-datascience.juanluis.safegraph_patterns",
    blobs,
    job_config_dict = job_config,
    bq_client = bq_client
)